
# Demonstration of PET reconstruction with SIRF
This demonstration shows how to use OSEM and implement a
(simplistic) gradient-ascent algorithm using SIRF.


Authors: Kris Thielemans and Evgueni Ovtchinnikov  
First version: 8th of September 2016  
Second Version: 17th of May 2018

CCP PETMR Synergistic Image Reconstruction Framework (SIRF)  
Copyright 2015 - 2017 Rutherford Appleton Laboratory STFC  
Copyright 2015 - 2018 University College London.

This is software developed for the Collaborative Computational
Project in Positron Emission Tomography and Magnetic Resonance imaging
(http://www.ccppetmr.ac.uk/).

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Initial set-up

In [ ]:
#%% make sure figures appears inline and animations works
%matplotlib notebook

In [ ]:
#%% Initial imports etc
import numpy
from numpy.linalg import norm
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os
import sys
import shutil
#import scipy
#from scipy import optimize
import sirf.STIR as pet
from sirf.Utilities import examples_data_path

In [ ]:
#%% some handy function definitions
def imshow(image, limits, title=''):
    """Usage: imshow(image, [min,max], title)"""
    plt.title(title)
    bitmap = plt.imshow(image)
    if len(limits)==0:
        limits = [image.min(), image.max()]
                
    plt.clim(limits[0], limits[1])
    plt.colorbar(shrink=.6)
    plt.axis('off')
    return bitmap

def make_positive(image_array):
    """truncate any negatives to zero"""
    image_array[image_array<0] = 0
    return image_array

def make_cylindrical_FOV(image):
    """truncate to cylindrical FOV"""
    filter = pet.TruncateToCylinderProcessor()
    filter.apply(image)

In [ ]:
#%% go to directory with input files
# adapt this path to your situation (or start everything in the relevant directory)
os.chdir(examples_data_path('PET'))

In [ ]:
#%% copy files to working folder and change directory to where the output files are
shutil.rmtree('working_folder/thorax_single_slice',True)
shutil.copytree('thorax_single_slice','working_folder/thorax_single_slice')
os.chdir('working_folder/thorax_single_slice')

## We will first create some simulated data from ground-truth images



In [ ]:
#%% Read in images
image = pet.ImageData('emission.hv')
image_array = image.as_array()*.05
image.fill(image_array)
mu_map = pet.ImageData('attenuation.hv')
mu_map_array = mu_map.as_array()

In [ ]:
#%% bitmap display of images
im_slice = image_array.shape[0]//2
plt.figure()
#plt.subplot(1,2,1)
imshow(image_array[im_slice,:,:,], [], 'emission image');
#plt.subplot(1,2,2)
#imshow(mu_map_array[im_slice,:,:,], [], 'attenuation image');

In [ ]:
#%% save max for future displays
cmax = image_array.max()*.6

In [ ]:
#%% create acquisition model
am = pet.AcquisitionModelUsingRayTracingMatrix()
# we will increate the number of rays used for every Line-of-Response (LOR) as an example
# (it is not required for the exercise of course)
am.set_num_tangential_LORs(5)
templ = pet.AcquisitionData('template_sinogram.hs')
am.set_up(templ,image);

In [ ]:
#%% simulate some data using forward projection
acquired_data=am.forward(image)
acquisition_array = acquired_data.as_array()

In [ ]:
#%% Display bitmaps of a middle sinogram
plt.figure()
imshow(acquisition_array[0,0,:,:,], [], 'Forward projection');

# Reconstruction via a SIRF reconstruction class
While you can write your own reconstruction algorithm by using `AcquisitionModel` etc, we also
provide a few reconstruction clases. We will show how to use these here.

## create the objective function

In PET, the iterative algorithms in SIRF rely on an objective function (i.e. the function to maximise).
In this case, this is the Poisson log-likelihood (without any priors)

In [ ]:
#%% create objective function
obj_fun = pet.make_Poisson_loglikelihood(acquired_data)
# We could set acquisition model but the default (ray-tracing) is in this case ok
# obj_fun.set_acquisition_model(am)
# we could also add a prior, but we will not do that here (although the rest of the exercise would still work)
#obj_fun.set_prior(prior)

## create OSMAPOSL reconstructor
This implements the Ordered Subsets Maximum A-Posteriori One Step Late
Since we are not using a penalty, or prior in this example, it
defaults to using MLEM, but we will modify it to OSEM

In [ ]:
recon = pet.OSMAPOSLReconstructor()
recon.set_objective_function(obj_fun)
recon.set_num_subsets(4)
num_iters=10
recon.set_num_subiterations(num_iters)

## Use this reconstructor!

In [ ]:
#%%  create initial image
# we could just use a uniform image but here we will create a disk with a different
# initial value (this will help the display later on)
init_image=image.clone()
init_image.fill(cmax / 4)
make_cylindrical_FOV(init_image)
# display
idata = init_image.as_array()
im_slice = idata.shape[0] // 2
plt.figure()
imshow(idata[im_slice,:,:],[0,cmax], 'initial image');

In [ ]:
#%% reconstruct the image 
reconstructed_image = init_image.clone()
# set up the reconstructor
recon.set_up(reconstructed_image)
# do actual recon
recon.reconstruct(reconstructed_image)

In [ ]:
#%% bitmap display of images
reconstructed_array = reconstructed_image.as_array()

plt.figure()
plt.subplot(1,2,1)
imshow(image_array[im_slice,:,:,], [0,cmax*1.2],'emission image')
plt.subplot(1,2,2)
imshow(reconstructed_array[im_slice,:,:,], [0,cmax*1.2], 'reconstructed image');


## For illustration, we do the same with noise in the data

In [ ]:
#%% Generate a noisy realisation of the data
noisy_array=numpy.random.poisson(acquisition_array).astype('float64')
print(' Maximum counts in the data: %d' % noisy_array.max())
# stuff into a new AcquisitionData object
noisy_data = acquired_data.clone()
noisy_data.fill(noisy_array);

In [ ]:
#%% Display bitmaps of the middle sinogram
plt.figure()
plt.subplot(1,2,1)
imshow(acquisition_array[0,im_slice,:,:,], [0,acquisition_array.max()], 'original')
plt.subplot(1,2,2)
imshow(noisy_array[0,im_slice,:,:,], [0,acquisition_array.max()], 'noisy');

In [ ]:
#%% reconstruct the noisy data
obj_fun.set_acquisition_data(noisy_data)
# We could save the data to file if we wanted to, but currently we don't.
# recon.set_output_filename_prefix('reconstructedImage_noisydata')

noisy_reconstructed_image = init_image.clone()
recon.reconstruct(noisy_reconstructed_image)

In [ ]:
#%% bitmap display of images
noisy_reconstructed_array = noisy_reconstructed_image.as_array()

plt.figure()
plt.subplot(1,2,1)
imshow(reconstructed_array[im_slice,:,:,], [0,cmax*1.2], 'no noise')
plt.subplot(1,2,2)
imshow(noisy_reconstructed_array[im_slice,:,:,], [0,cmax*1.2], 'with noise');

# Taking control of the iteration process
We will now show how to run each sub-iteration from in Python, as opposed to
letting the reconstructor do all sub-iterations at once.

In [ ]:
#%% run same reconstruction but saving images and objective function values every sub-iteration
num_subiters = 64
# create an image object that will be updated during the iterations
current_image = init_image.clone()
# create an array to store the values of the objective function at every
# sub-iteration (and fill in the first)
osem_objective_function_values = [obj_fun.value(current_image)]
# create an ndarray to store the images at every sub-iteration
all_osem_images = numpy.ndarray(shape=(num_subiters + 1,) + idata.shape)
all_osem_images[0,:,:,:] = current_image.as_array()
# do the loop
for i in range(1, num_subiters+1):
    recon.update(current_image)
    # store results
    obj_fun_value = obj_fun.value(current_image)
    osem_objective_function_values.append(obj_fun_value)
    all_osem_images[i,:,:,:] =  current_image.as_array();

## Make some plots with these results

In [ ]:
#%% define a function for plotting images and the updates
def plot_progress(all_images, title, subiterations = []):
    if len(subiterations) == 0:
        num_subiters = all_images[0].shape[0] - 1
        subiterations = range(1, num_subiters + 1)
    num_rows = len(all_images)
    #plt.close('all')
    for i in subiterations:
        plt.figure(i)
        for r in range(num_rows):
            plt.subplot(num_rows,2,2 * r + 1)
            imshow(all_images[r][i,im_slice,:,:], [0,cmax], '%s at %d' % (title[r], i))
            plt.subplot(num_rows,2,2*r+2)
            imshow(all_images[r][i,im_slice,:,:]-all_images[r][i - 1,im_slice,:,:],[-cmax*.1,cmax*.1], 'update')
        #plt.pause(.05)
        plt.show()

In [ ]:
#%% now call this function to see how we went along
# note that in the notebook interface, this might create a box with a vertical slider
subiterations = (1,2,4,8,16,32,64)
plot_progress([all_osem_images], ['OSEM'],subiterations)

In [ ]:
#%% plot objective function values
plt.figure()
#plt.plot(subiterations, [ osem_objective_function_values[i] for i in subiterations])
plt.plot(osem_objective_function_values)
plt.title('Objective function values')
plt.xlabel('sub-iterations');

The above plot seems to indicate that (OS)EM converges to a stable value of the
log-likelihood very quickly. However, as we've seen, the images are still changing.

Convince yourself that the likelihood is still increasing (either by zooming into the figure, or by using `plt.ylim`).

We can compute some simple ROI values as well. Let's plot those.

In [ ]:
#%% ROI
ROI_lesion = all_osem_images[:,(im_slice,), 65:70, 40:45]
ROI_lung = all_osem_images[:,(im_slice,), 75:80, 45:50]

ROI_mean_lesion = ROI_lesion.mean(axis=(1,2,3))
ROI_std_lesion = ROI_lesion.std(axis=(1,2,3))

ROI_mean_lung = ROI_lung.mean(axis=(1,2,3))
ROI_std_lung = ROI_lung.std(axis=(1,2,3))

plt.figure()
#plt.hold('on')
plt.subplot(1,2,1)
plt.plot(ROI_mean_lesion,'k',label='lesion')
plt.plot(ROI_mean_lung,'r',label='lung')
plt.legend()
plt.title('ROI mean')
plt.xlabel('sub-iterations')
plt.subplot(1,2,2)
plt.plot(ROI_std_lesion, 'k',label='lesion')
plt.plot(ROI_std_lung, 'r',label='lung')
plt.legend()
plt.title('ROI standard deviation')
plt.xlabel('sub-iterations');

The above plots indicate that the log-likelihood is not a very sensitive
measure of changes in the image. This an illustration that image reconstruction
is an ill-conditioned inverse problem.

# Implement gradient ascent and compare with OSEM
Here we will implement a simple version of Gradient Ascent using SIRF functions.We will use
the SIRF capability to return the gradient of the objective function directly.

Gradient ascent (GA) works by updating the image in the direction of the gradient

    new_image = current_image + step_size * gradient

Here we will use a fixed step-size and use "truncation" to enforce
non-negativity of the image

In [ ]:
#%% Define some variables to perform gradient ascent for a few (sub)iterations
num_subiters = 32
# relative step-size
tau = .3

# set initial image and store it
# also store the value of the objective function for plotting
current_image = init_image.clone()
GA_objective_function_values = [obj_fun.value(current_image)]
# create an array with all reconstruct images for plotting
idata = current_image.as_array()
all_images = numpy.ndarray(shape=(num_subiters + 1,) + idata.shape)
all_images[0,:,:,:] =  idata;

In [ ]:
#%% perform GA iterations
# executing this cell might take a while
for i in range(1, num_subiters+1):  
    # obtain gradient for subset 0
    # with current settings, this means we will only use the data of that subset
    # (gradient ascent with subsets is too complicated for this demo)
    grad = obj_fun.gradient(current_image, 0)
    grad_array = grad.as_array()
    
    # compute step-size as relative to current image-norm
    step_size = tau * norm(idata) / norm(grad_array)
    
    # perform gradient ascent step and truncate to positive values
    idata = make_positive(idata + step_size*grad_array)
    current_image.fill(idata)

    # compute objective function value for plotting, and write some diagnostics
    obj_fun_value = obj_fun.value(current_image)
    GA_objective_function_values.append(obj_fun_value)
    all_images[i,:,:,:] = idata;

In [ ]:
#%% Plot objective function values
plt.figure()
#plt.hold('on')
plt.title('Objective function value vs subiterations')
plt.plot(GA_objective_function_values,'b')
plt.plot(osem_objective_function_values,'r')
plt.legend(('gradient ascent', 'OSEM'),loc='lower right');

In [ ]:
#%% compare GA and OSEM images
plot_progress([all_images, all_osem_images], ['GA' ,'OSEM'],[2,4,8,16,32])

The above implementation used a fixed (relative) step-size. Experiment with different values for `tau` and see how that influences convergence.

Steepest gradient ascent will include a line search to estimate the step size. There is a demo
in the SIRF code on this. You can [find the code here as well](https://github.com/CCPPETMR/SIRF/blob/master/examples/Python/PET/steepest_ascent.py). You could implement this here.